# Navigator

In [209]:
import json
import requests
import pandas as pd

from joblib import Parallel, delayed
from tqdm.notebook import tqdm

## Get All Organisations

In [32]:
with open('all_services.json', 'rb') as file:
    org_json = json.load(file)

In [33]:
all_orgs = []

for d in org_json:
    all_orgs.append(d['t'][0]['v'])

In [34]:
len(all_orgs)

373

In [213]:
pd.DataFrame(all_orgs, columns=['ncss_member'])

ncss_member
0                        365 Cancer Prevention Society
1                                     *SCAPE Co., Ltd.
2    Abilities Beyond Limitations And Expectations ...
3                                     ADAM Association
4                             Adullam Life Counselling
5                        Adventist Home for the Elders
6          Adventist Nursing and Rehabilitation Centre
7                Agape Counselling And Training Centre
8                                     Agape Little Uni
9                                            Aidha Ltd
10                                         Ain Society
11                                           ALIFE Ltd
12                             Alive Community Network
13                                     All Saints Home
14                     Alzheimer's Disease Association
15           American Women's Association of Singapore
16                       AMKFSC Community Services Ltd
17                         Aoxiang Counselling Service
18          Apex Day Rehabilitation Centre For Elderly
19                                  Apex Harmony Lodge
20                    Arc Children's Centre Co Limited
21               Art Therapists' Association Singapore
22                        Asia Philanthropy Circle Ltd
23        Assemblies of God Community Services Society
24                                      Assisi Hospice
25          Association For Persons With Special Needs
26                 Association Of Muslim Professionals
27        Association Of Women For Action And Research
28    Australian & New Zealand Association (Singapore)
29                      Autism Association (Singapore)
..                                                 ...
343              Thye Hua Kwan Moral Charities Limited
344                        Thye Hua Kwan Moral Society
345                   TOUCH Community Services Limited
346                      TOUCH Family Services Limited
347                        TRANS Family Service Centre
348                              TRANS Family Services
349                 Transient Workers Count Too (TWC2)
350                                      Trybe Limited
351                                    Tsao Foundation
352                       Tung Ling Community Services
353                                 Turning Point, The
354                             United Medicare Centre
355            United World College of South East Asia
356                    Very Special Arts Singapore Ltd
357         Vipassana International Centre (Singapore)
358                          Viriya Community Services
359           Viva Foundation for Children with Cancer
360                         Wan Min Community Services
361                 WE CARE Community Services Limited
362                               Wicare Support Group
363                                     Willing Hearts
364                            Woodlands Social Centre
365                            Xin Yuan Community Care
366                                    Yayasan MENDAKI
367                           Yellow Ribbon Fund (YRF)
368                                Yong-en Care Centre
369     Young Men's Christian Association Of Singapore
370   Young Women's Christian Association Of Singapore
371                   Youth Guidance Outreach Services
372                             Zion Home for the Aged

[373 rows x 1 columns]

# Get All Programmes
1. Copy bash Curl request
2. Paste into Excel
3. Excel automatically structures requests based on:
    1. Start: y=0
    2. End: y=44482
    3. No. of pixels per row = 23
    4. No. of elements = 1934
4. Create bash script
5. Run and save to results.txt

In [106]:
all_lines = []
with open('resources/Navigator Data/results.txt', 'rb') as file:
    for line in file:
        all_lines.append(line)

In [109]:
from bs4 import BeautifulSoup
all_html = []
for line in all_lines:
    all_html.append(eval(eval(line.decode('utf-8').replace('false', 'False'))['vqlCmdResponse']['cmdResultList'][0]['commandReturn']['tooltipText'])['htmlTooltip'])

In [161]:
all_progs = []
all_desc = []
all_target = []

nonstandard = []

for l, line in enumerate(all_html):
    soup = BeautifulSoup(line)
    div_list = soup.find_all('div')
    if len(div_list) != 5:
        nonstandard.append(l)
    current = 'prog'
    prog = ''
    desc = ''
    target = ''
    for i, div in enumerate(div_list):
                
        if i == 0:
            current = 'prog'
        else:
            if prog != '' and desc == '' and target == '' and div.text == '':
                current = 'desc'
            elif prog != '' and desc != '' and target == '' and div.text[:14] == 'Target Clients':
                current = 'target'
        
        if current == 'prog':
            if prog == '':
                prog = div.text
            else:
                prog += ' ' + div.text
        elif current == 'desc':
            if desc == '':
                desc = div.text
            else:
                desc += ' ' + div.text
        elif current == 'target':
            if target == '':
                target = div.text
            else:
                target += ' ' + div.text
        
    # Append all
    all_progs.append(prog)
    all_desc.append(desc)
    all_target.append(target)

In [173]:
import pandas as pd
df = pd.DataFrame({
    'programmes': all_progs,
    'desc': all_desc,
    'target': all_target})

df['target'] = df.target.str.replace('Target Clients : ', '')

# Mendarki youth programmes have no info
df.loc[1000, 'desc'] = ''
df.loc[1000, 'target'] = ''

# df.to_csv('resources/Navigator Data/ncss_programmes.csv', index=False)

In [177]:
with open('resources/Navigator Data/family-services-geojson.geojson', 'rb') as file:
    fsc = json.load(file)

In [204]:
all_feats = []
for feat in fsc['features']:
    keys = [x.find('th').text for x in BeautifulSoup(feat['properties']['Description']).find_all('tr')[1:]]
    values = [x.find('td').text for x in BeautifulSoup(feat['properties']['Description']).find_all('tr')[1:]]
    all_feats.append({k: v for k,v in zip(keys, values)})

In [208]:
all_feats = pd.DataFrame(all_feats)

ADDRESSBLOCKHOUSENUMBER ADDRESSBUILDINGNAME ADDRESSFLOORNUMBER  \
0                                                                   
1                                                                   
2                                                                   
3                                                                   
4                                                                   
5                                                                   
6                                                                   
7                                                                   
8                                                                   
9                                                                   
10                                                                  
11                                                                  
12                                                                  
13                                                                  
14                                                                  
15                                                                  
16                                                                  
17                                                                  
18                                                                  
19                                                                  
20                                                                  
21                                                                  
22                                                                  
23                                                                  
24                                                                  
25                                                                  
26                                                                  
27                                                                  
28                                                                  
29                                                                  
30                                                                  
31                                                                  
32                                                                  
33                                                                  
34                                                                  
35                                                                  
36                                                                  
37                                                                  
38                                                                  
39                                                                  
40                                                                  
41                                                                  
42                                                                  
43                                                                  
44                                                                  
45                                                                  
46                                                                  
47                                                                  
48                                                                  
49                                                                  

   ADDRESSPOSTALCODE                                  ADDRESSSTREETNAME  \
0             090027                  Blk 27 Telok Blangah Way #01-1018   
1             050004                            Blk 4 Sago Lane #03-101   
2             460426                   Blk 426 Bedok North Road #01-515   
3             560230                  Blk 230 Ang Mo Kio Ave 3 #01-1264   
4             560643                  Blk 643 Ang Mo Kio Ave 5 #01-3001   
5             321107             

In [24]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [87]:
all_lines = []
with open('resources/Scrape Programmes/results.txt', 'rb') as file:
    for line in file:
        all_lines.append(line)

In [136]:
all_links = []
all_orgs = []
for i, line in enumerate(all_lines):
    links_list = eval(eval(line.decode('utf-8').replace('false', 'False'))['vqlCmdResponse']['cmdResultList'][0]['commandReturn']['tooltipText'])['actions']['commandItems']
    text=''
    for link in links_list:
        
        if link['name'] == 'Click here for more information.':
            text = link['command']
            text = re.sub('.*url="', '', text)
            text = re.sub('" visual-id-pres.*', '', text)
            text = text.strip()
            if text == '':
                all_links.append('')
            else:
                all_links.append(text)
        
        if i in [266, 269, 274, 436, 894, 1868]:
            all_links.append('')
        
        if link['name'] == 'Click here for directions':
            text = link['command']
            text = re.sub('.*maps\?q=', '', text)
            text = re.sub('" visual-id-pres.*', '', text)
            text = text.strip()
            if text == '':
                all_orgs.append('')
            else:
                all_orgs.append(text)
                

In [141]:
df = pd.read_csv('resources/Scrape Programmes/ncss_programmes.csv')
df['link'] = all_links
df['link'] = df.link.str.replace('.sg/.*', '.sg/')
df['link'] = df.link.str.replace('.org/.*', '.org/')
df['link'] = df['link'].astype(str)
df.loc[~df.link.str.contains('facebook'), 'link'] = df.loc[~df.link.str.contains('facebook'), 'link'].str.replace('.com/.*', '.com/')

df['org'] = all_orgs
df.to_csv('resources/Scrape Programmes/ncss_programmes.csv', index=False)